In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_url

In [ ]:
# 2017 properties
# keep all the features
# only include last transaction (no duplicate property IDs)
# zestimate error
# data of transaction
# only include properties that have a latitude and longitude

url = get_url("zillow")

sql = """
select * from properties_2017
join predictions_2017 using(parcelid)
left join airconditioningtype using(airconditioningtypeid)
left join architecturalstyletype using(architecturalstyletypeid)
left join buildingclasstype using(buildingclasstypeid)
left join heatingorsystemtype using(heatingorsystemtypeid)
left join propertylandusetype using(propertylandusetypeid)
left join storytype using(storytypeid)
left join typeconstructiontype using(typeconstructiontypeid)
where latitude IS NOT NULL
and longitude IS NOT NULL
"""

In [2]:
# If you want to cache the raw acquired input, do this:
# df = pd.read_sql(sql, url)
# df.to_csv("zillow.csv")
# df = pd.read_csv("zillow.csv")

In [3]:
df.head()

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (65) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


## Directions said to remove duplicate parcelids
- It looks like all the joins, a subquery, and maybe a temp table. 
- But, I'm on a time constraint, so let's check the cost vs the benefit.
- All transactions is 77614, number of unique parcelids is 77414, so for 200 transactions, I have duplicates.
- This is a low cost, high return item to skip. I'll come back and clean this up later, but for 200/77000 observations, I have higher value work to do first.

```sql
# 77,614
select count(*)
from predictions_2017;
```

vs
```sql
# 77,414
select count(distinct(parcelid))
from predictions_2017;
```

> Sometimes, knowing what *not* to do and when to *not* do it is as important as knowing how to do that thing.

In [4]:
# quantify the downside
200 / 77614

0.0025768546911639654

## Think Critically and Problem Solve
- What's the problem this instruction is trying to solve?
- What's the goal of the remove duplicates directions? 
- Noise removal is good. We want to optimize signal to noise.
- Did the instructions come from someone who knew 
- How long did it take to remove duplicate parcelids propertly?
- Is .0026 OK to skip for now? 
- Yes, especially if it let's the river keep flowing instead of damming it up for a few more/less raindrops drops.
- What's the problem behind the problem?

In [5]:
df.head()

,Unnamed: 0,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,...,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,...,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,...,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,...,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,...,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,10887214,1970746,...,4,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [6]:
# Obtain the counts of all the nulls and sort
null_counts = df.isnull().sum().sort_values(ascending=False)

# What are the half of the columns w/ the most nulls
# series[start:end]
null_counts.head()

buildingclassdesc       77565
buildingclasstypeid     77565
finishedsquarefeet13    77538
storytypeid             77530
basementsqft            77530
dtype: int64

In [7]:
# Show the lower nulls
null_counts.tail(30)

finishedsquarefeet12            3656
regionidcity                    1472
calculatedbathnbr                616
fullbathcnt                      616
yearbuilt                        270
censustractandblock              247
calculatedfinishedsquarefeet     201
structuretaxvaluedollarcnt       115
regionidzip                       50
taxamount                          5
landtaxvaluedollarcnt              2
taxvaluedollarcnt                  1
id.1                               0
parcelid                           0
propertylandusedesc                0
propertylandusetypeid              0
roomcnt                            0
regionidcounty                     0
transactiondate                    0
rawcensustractandblock             0
id                                 0
fips                               0
logerror                           0
bathroomcnt                        0
bedroomcnt                         0
propertycountylandusecode          0
longitude                          0
l

In [8]:
# Let's figure out how much data is missing where
def nulls_by_col(df):
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    pct_missing = num_missing / rows
    cols_missing = pd.DataFrame({'number_missing_rows': num_missing, 'percent_rows_missing': pct_missing})
    return cols_missing

In [9]:
nulls_by_column = nulls_by_col(df)
nulls_by_column.sort_values(by="percent_rows_missing", ascending=False, inplace=True)
nulls_by_column.head(10)

,number_missing_rows,percent_rows_missing
buildingclassdesc,77565,0.999807
buildingclasstypeid,77565,0.999807
finishedsquarefeet13,77538,0.999459
storytypeid,77530,0.999356
storydesc,77530,0.999356
basementsqft,77530,0.999356
yardbuildingsqft26,77510,0.999098
fireplaceflag,77408,0.997783
architecturalstyletypeid,77373,0.997332
architecturalstyledesc,77373,0.997332


In [10]:
def nulls_by_row(df):
    num_cols_missing = df.isnull().sum(axis=1)
    pct_cols_missing = df.isnull().sum(axis=1)/df.shape[1]*100
    rows_missing = pd.DataFrame({'num_cols_missing': num_cols_missing, 'pct_cols_missing': pct_cols_missing}).reset_index().groupby(['num_cols_missing','pct_cols_missing']).count().rename(index=str, columns={'index': 'num_rows'}).reset_index()
    return rows_missing 

null_rows = nulls_by_row(df)
null_rows.sort_values(by="pct_cols_missing", ascending=False, inplace=True)
null_rows.head(10)

,num_cols_missing,pct_cols_missing,num_rows
25,48,68.57142857142857,3
24,47,67.14285714285714,3
23,46,65.71428571428571,5
22,45,64.28571428571429,50
21,44,62.857142857142854,79
20,43,61.42857142857143,29
19,42,60.0,27
18,41,58.57142857142858,29
17,40,57.14285714285714,230
16,39,55.714285714285715,285


## Prepare Goals
- How do we define a single unit? No land/lot, duplexes, commercial
- Remove any properties that are likely something other than single unit. 
    - Do not purely filter by unitcnt 
    - Goal is to reduce single units that are falsely removed as something else.
   
- Ideas:
    - filter out 0 bedroom and 0 bathroom properties.
    - room count greater than 1
    - squarefootage more than 400
    - filter by propertylandusetype
        - keep:
            - 260, residential general
            - 261, single family residential
            - 262, rural residence
            - 279 inferred single family residential

In [11]:
# .isin([collection])
df = df[df.propertylandusetypeid.isin([260, 261, 262, 279])]

In [12]:
# Only retain homes with a positive number of bedrooms and bathrooms
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0)]
df.shape

(52300, 70)

In [13]:
df.unitcnt.value_counts()

1.0    33783
2.0       28
4.0        3
3.0        1
Name: unitcnt, dtype: int64

In [14]:
# 18485 null unit counts. They're probably unit counts of 1.
# It's more likely the paperwork for a single unit didn't have a value
df.unitcnt.isnull().sum()

18485

In [15]:
df.unitcnt = df.unitcnt.fillna(1.0)
df.shape

(52300, 70)

In [16]:
# How many unitcounts are not 1?
(df.unitcnt != 1.0).sum()

32

In [17]:
# Filter the duplex, triplex, etc...
df = df[df.unitcnt == 1.0]

In [18]:
# Drop logically unnecessary columns
# Unnamed:0
# id
# id.1
df = df.drop(columns=["Unnamed: 0", "id", "id.1"])

In [19]:
def handle_missing_values(df, prop_required_column = .60, prop_required_row = .60):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [20]:
df = handle_missing_values(df)

In [21]:
# Columns to drop b/c they're ids of descriptions
# propertylandusetypeid
# heatingorsystemtypeid
df = df.drop(columns=["propertylandusetypeid", "heatingorsystemtypeid"])

In [22]:
df.isnull().sum().sort_values(ascending=False)

buildingqualitytypeid           18571
propertyzoningdesc              18510
heatingorsystemdesc             18369
regionidcity                     1029
lotsizesquarefeet                 351
finishedsquarefeet12              166
censustractandblock               109
structuretaxvaluedollarcnt         71
yearbuilt                          40
regionidzip                        23
calculatedbathnbr                  16
fullbathcnt                        16
calculatedfinishedsquarefeet        8
taxamount                           4
landtaxvaluedollarcnt               1
taxvaluedollarcnt                   1
latitude                            0
bedroomcnt                          0
fips                                0
bathroomcnt                         0
propertylandusedesc                 0
longitude                           0
propertycountylandusecode           0
rawcensustractandblock              0
regionidcounty                      0
roomcnt                             0
unitcnt     

In [23]:
# propertyzoningdesc = Description of the allowed land uses (zoning) for that property
# I'll drop this, b/c we're already filtering for single unit residential.

In [24]:
df.heatingorsystemdesc.isnull().sum()

18369

In [25]:
df.heatingorsystemdesc.value_counts(dropna=False)

Central       20723
NaN           18369
Floor/Wall    12542
Forced air      517
Solar            85
None             16
Baseboard         7
Radiant           6
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

In [26]:
# b/c these properties are in southern california 
df.heatingorsystemdesc = df.heatingorsystemdesc.fillna("None")

In [27]:
df = df.drop(columns=["propertyzoningdesc"])

In [28]:
# Looks like calculatedbathrbr is equivalent to bathroomcnt
(df.calculatedbathnbr == df.bathroomcnt).sum() / len(df)

0.9996938853600673

In [29]:
df = df.drop(columns=["calculatedbathnbr"])

In [30]:
df.isna().sum().sort_values(ascending=False).head(10)

buildingqualitytypeid           18571
regionidcity                     1029
lotsizesquarefeet                 351
finishedsquarefeet12              166
censustractandblock               109
structuretaxvaluedollarcnt         71
yearbuilt                          40
regionidzip                        23
fullbathcnt                        16
calculatedfinishedsquarefeet        8
dtype: int64

## It's a Good Time to Split the Data 
- Split the df into train/validate/test
- Impute values from `train`, then apply them to validate and test sets
- Keep our out of sample datasets out of sample

In [31]:
from sklearn.model_selection import train_test_split

train_and_validate, test = train_test_split(df, train_size=.8, random_state=123)
train, validate = train_test_split(train_and_validate, random_state=123)

In [32]:
# Categorical/Discrete columns to use mode to replace nulls

cols = [
    "buildingqualitytypeid",
    "regionidcity",
    "regionidzip",
    "yearbuilt",
    "regionidcity",
    "censustractandblock"
]

for col in cols:
    mode = int(train[col].mode()) # I had some friction when this returned a float (and there were no decimals anyways)
    train[col].fillna(value=mode, inplace=True)
    validate[col].fillna(value=mode, inplace=True)
    test[col].fillna(value=mode, inplace=True)

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [33]:
train.isnull().sum().sort_values(ascending=False).head(10)

lotsizesquarefeet               195
finishedsquarefeet12             99
structuretaxvaluedollarcnt       38
fullbathcnt                       8
calculatedfinishedsquarefeet      4
taxamount                         2
landtaxvaluedollarcnt             1
taxvaluedollarcnt                 1
propertylandusedesc               0
propertycountylandusecode         0
dtype: int64

In [34]:
# Continuous valued columns to use median to replace nulls

cols = [
    "structuretaxvaluedollarcnt",
    "taxamount",
    "taxvaluedollarcnt",
    "landtaxvaluedollarcnt",
    "structuretaxvaluedollarcnt",
    "finishedsquarefeet12",
    "calculatedfinishedsquarefeet",
    "fullbathcnt",
    "lotsizesquarefeet"
]


for col in cols:
    median = train[col].median()
    train[col].fillna(median, inplace=True)
    validate[col].fillna(median, inplace=True)
    test[col].fillna(median, inplace=True)

In [39]:
train.isnull().sum().sort_values(ascending=False)

propertylandusedesc             0
regionidcity                    0
bathroomcnt                     0
bedroomcnt                      0
buildingqualitytypeid           0
calculatedfinishedsquarefeet    0
finishedsquarefeet12            0
fips                            0
fullbathcnt                     0
latitude                        0
longitude                       0
lotsizesquarefeet               0
propertycountylandusecode       0
rawcensustractandblock          0
regionidcounty                  0
heatingorsystemdesc             0
regionidzip                     0
roomcnt                         0
unitcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
censustractandblock             0
logerror                        0
transactiondate                 0
parcelid                        0
dtype: int64

In [40]:
# How many nulls left in the dataset?
train.isnull().sum().sort_values(ascending=False).sum()

0

# One Weird Trick To Get Things Done

> Keep Moving

- Are you in a time crunch?
- Will the database be changing as you do analysis?
- Is this a one-shot data preparation?
- Does this need to be the cleanest DRYest Python you've ever written?
- Does your MVP for the project need a wrangle.py that will stand the test of time for a million users?
- Or do you need some CSVs super quick?

![minimum viable product diagram](https://www.educati.ch/wp-content/uploads/2018/11/MVP_HQ4-1024x724.jpg)

## Then Write The Output to Disk and Move Forward
- Write `train.to_csv("clean_zillow_train.csv")`
- Write `validate.to_csv("clean_zillow_validate.csv")`
- Write `test.to_csv("clean_zillow_test.csv")`

In [36]:
train.to_csv("zillow_train.csv")
validate.to_csv("zillow_validate.csv")
test.to_csv("zillow_test.csv")